# Fine-tuning XLM-R on CodiEsp-P

In this notebook, following a multi-label sequence classification approach, the XLM-R model is fine-tuned on both the training and development sets of the CodiEsp-P corpus. Additionally, the predictions made by the model on the test set are saved, in order to futher evaluate the clinical coding performance of the model (see `results/CodiEsp-P/Evaluation.ipynb`).

In [1]:
import tensorflow as tf

# Auxiliary components
import sys
sys.path.append("..")
from nlp_utils import *

# XLM-R tokenizer
from transformers import XLMRobertaTokenizer
import sentencepiece_pb2
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
spt = sentencepiece_pb2.SentencePieceText()

# Hyper-parameters
text_col = "raw_text"
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS = 41
LR = 3e-5

random_seed = 0
tf.random.set_seed(random_seed)

## Load text

Firstly, all text files from training and development CodiEsp corpora are loaded in different dataframes.

Also, CIE-Procedimiento codes are loaded.

In [2]:
corpus_path = "../datasets/codiesp_v4/"

### Training corpus

In [3]:
%%time
train_path = corpus_path + "train/text_files/"
train_files = [f for f in os.listdir(train_path) if os.path.isfile(train_path + f)]
train_data = load_text_files(train_files, train_path)
df_text_train = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in train_files], 'raw_text': train_data})

CPU times: user 7.95 ms, sys: 4 ms, total: 11.9 ms
Wall time: 11.5 ms


In [4]:
df_text_train.shape

(500, 2)

In [5]:
df_text_train.head()

,doc_id,raw_text
0,S0365-66912006000600011-1,Un varón de 13 años es remitido para valoració...
1,S1139-13752009000200010-2,Paciente de 42 años diagnosticado de pancoliti...
2,S1130-05582017000100037-1,"Varón de 72 años, sin antecedentes médicos de ..."
3,S1139-76322016000300015-1,Lactante de ocho días cuyos padres consultan p...
4,S0211-69952011000100019-1,Mujer de 47 años de edad con antecedentes de g...


In [6]:
df_text_train.raw_text[0]

'Un varón de 13 años es remitido para valoración oftalmológica por mala visión. Fenotípicamente era un niño de talla corta con una estatura de 133 cm, braquimorfia y braquidactilia en las cuatro extremidades.\nEl paciente presentaba un error refractivo corregido de -13,00 -6,50 a 1º en el ojo derecho y de -16,00-6,25 a 179º en el izquierdo. Con dicha corrección alcanzaba una agudeza visual de 0,4 y 0,2 respectivamente. No existía diplopía monocular ni hallazgos en la motilidad ocular extrínseca e intrínseca.\nEl diámetro corneal horizontal era de 12,0 mm en ambos ojos y la paquimetría de 613 y 611 micras respectivamente. La cámara anterior era estrecha, apreciándose iridofacodonesis bilateral. Se evidenció microesferofaquia con desplazamiento anterior de ambos cristalinos dentro de la cámara posterior.\nLa presión intraocular era de 20 mmHg bilateralmente. Gonioscópicamente se apreció un ángulo estrecho simétrico en ambos ojos grado II según Schaffer.\nLa exploración mediante topógrafo

We also load the CIE-Procedimiento codes table:

In [7]:
df_codes_train = pd.read_table(corpus_path + "train/trainP.tsv", sep='\t', header=None)

In [8]:
df_codes_train.columns = ["doc_id", "code"]

In [9]:
df_codes_train.shape

(1550, 2)

In [10]:
df_codes_train.head()

,doc_id,code
0,S0004-06142005000700014-1,bw03zzz
1,S0004-06142005000700014-1,3e02329
2,S0004-06142005000700014-1,bw40zzz
3,S0004-06142005000700014-1,bv44zzz
4,S0004-06142005000700014-1,bn20


In [11]:
len(set(df_codes_train["doc_id"]))

435

### Development corpus

In [12]:
%%time
dev_path = corpus_path + "dev/text_files/"
dev_files = [f for f in os.listdir(dev_path) if os.path.isfile(dev_path + f)]
dev_data = load_text_files(dev_files, dev_path)
df_text_dev = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in dev_files], 'raw_text': dev_data})

CPU times: user 6.34 ms, sys: 153 µs, total: 6.49 ms
Wall time: 6.18 ms


In [13]:
df_text_dev.shape

(250, 2)

In [14]:
df_text_dev.head()

,doc_id,raw_text
0,S1130-63432016000600013-1,"Varón de 75 años, con antecedentes de hiperuri..."
1,S0365-66912003000600010-1,"Paciente de 33 años de edad, gestante de 34 se..."
2,S0211-69952012000200030-1,Mujer de 67 años con múltiples factores de rie...
3,S0365-66912004000900009-1,Paciente de 55 años que acudió a urgencias por...
4,S1139-76322016000300016-2,"Lactante de 1 mes y 29 días, sin antecedentes ..."


In [15]:
df_text_dev.raw_text[0]

'Varón de 75 años, con antecedentes de hiperuricemia en tratamiento con Alopurinol que ingresa para realización de resección transuretral de próstata.\nPostoperatorio inmediato sin incidencias con tratamiento con Pantoprazol, Ciprofloxacino, Paracetamol, Enantyum y Alopurinol. Al cuarto día de postoperatorio presenta mareos, temblor con componente mioclónico en extremidades y tronco e incapacidad para caminar, sin verse alteraciones analíticas. En esta situación se pauta Rivotril y se suspende el tratamiento con Ciprofloxacino, desapareciendo la clínica mioclónica y mejorando el estado del paciente, por lo que se decide el alta hospitalaria.\n\n'

We also load the CIE-Procedimiento codes table:

In [16]:
df_codes_dev = pd.read_table(corpus_path + "dev/devP.tsv", sep='\t', header=None)

In [17]:
df_codes_dev.columns = ["doc_id", "code"]

In [18]:
df_codes_dev.shape

(817, 2)

In [19]:
df_codes_dev.head()

,doc_id,code
0,S0004-06142005000900016-1,bt41zzz
1,S0004-06142005000900016-1,ct13
2,S0004-06142005001000011-1,3e1m39z
3,S0004-06142005001000011-1,0tcb
4,S0004-06142005001000011-1,bt02


In [20]:
len(set(df_codes_dev["doc_id"]))

222

We join the training and development CodiEsp codes dataframes together:

In [21]:
df_codes_train_dev = pd.concat([df_codes_train, df_codes_dev])

In [22]:
df_codes_train_dev.shape

(2367, 2)

In [23]:
df_codes_train_dev.head()

,doc_id,code
0,S0004-06142005000700014-1,bw03zzz
1,S0004-06142005000700014-1,3e02329
2,S0004-06142005000700014-1,bw40zzz
3,S0004-06142005000700014-1,bv44zzz
4,S0004-06142005000700014-1,bn20


## Creating corpora of annotated sentences

Leveraging the information available for the named-entity-recognition and normalization (NER-N) CodiEsp-X task, we create both a training and a development corpus of annotated sentences with CIE-Procedimiento codes.

Firstly, we pre-process the NER-N precedure-codes annotations available for both the training and development corpora.

In [24]:
# Training corpus

In [25]:
%%time

codiesp_x_train = pd.read_table(corpus_path + "train/trainX.tsv", sep='\t', header=None)

CPU times: user 9.52 ms, sys: 99 µs, total: 9.62 ms
Wall time: 8.98 ms


In [26]:
codiesp_x_train.columns = ["doc_id", "type", "code", "word", "location"]

In [27]:
codiesp_x_train.shape

(9181, 5)

In [28]:
codiesp_x_train.head()

,doc_id,type,code,word,location
0,S0004-06142005000700014-1,PROCEDIMIENTO,bw03zzz,Rx tórax,2163 2171
1,S0004-06142005000700014-1,PROCEDIMIENTO,3e02329,Estreptomicina intramuscular,2787 2801;2810 2823
2,S0004-06142005000700014-1,DIAGNOSTICO,n44.8,teste derecho aumentado de tamaño,1343 1376
3,S0004-06142005000700014-1,DIAGNOSTICO,z20.818,exposición a Brucella,594 615
4,S0004-06142005000700014-1,DIAGNOSTICO,r60.9,edemas,1250 1256


In [29]:
codiesp_x_train = codiesp_x_train[codiesp_x_train["type"] == "PROCEDIMIENTO"]

In [30]:
codiesp_x_train.shape

(1972, 5)

In [31]:
df_codes_train_ner = process_ner_labels(codiesp_x_train).sort_values(["doc_id", "start", "end"])

In [32]:
df_codes_train_ner.head()

,doc_id,type,code,word,start,end
0,S0004-06142005000700014-1,PROCEDIMIENTO,bw03zzz,Rx tórax,2163,2171
3,S0004-06142005000700014-1,PROCEDIMIENTO,bw40zzz,Ecografía abdominal,2173,2192
5,S0004-06142005000700014-1,PROCEDIMIENTO,bn20,TAC craneal,2194,2205
4,S0004-06142005000700014-1,PROCEDIMIENTO,bv44zzz,Ecografía testicular,2287,2307
1,S0004-06142005000700014-1,PROCEDIMIENTO,3e02329,Estreptomicina intramuscular,2787,2801


In [33]:
df_codes_train_ner.shape

(2769, 6)

In [34]:
# Development corpus

In [35]:
%%time

codiesp_x_dev = pd.read_table(corpus_path + "dev/devX.tsv", sep='\t', header=None)

CPU times: user 5.37 ms, sys: 85 µs, total: 5.46 ms
Wall time: 5.13 ms


In [36]:
codiesp_x_dev.columns = ["doc_id", "type", "code", "word", "location"]

In [37]:
codiesp_x_dev.shape

(4477, 5)

In [38]:
codiesp_x_dev.head()

,doc_id,type,code,word,location
0,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,307 316;348 361
1,S0004-06142005000900016-1,PROCEDIMIENTO,ct13,gammagrafía renal,739 756
2,S0004-06142005000900016-1,DIAGNOSTICO,q62.11,estenosis en la unión pieloureteral derecha,540 583
3,S0004-06142005000900016-1,DIAGNOSTICO,n28.89,ectasia pielocalicial,326 347
4,S0004-06142005000900016-1,DIAGNOSTICO,n39.0,infecciones del tracto urinario,198 229


In [39]:
codiesp_x_dev = codiesp_x_dev[codiesp_x_dev["type"] == "PROCEDIMIENTO"]

In [40]:
codiesp_x_dev.shape

(1046, 5)

In [41]:
df_codes_dev_ner = process_ner_labels(codiesp_x_dev).sort_values(["doc_id", "start", "end"])

In [42]:
df_codes_dev_ner.head()

,doc_id,type,code,word,start,end
0,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,307,316
1,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,348,361
2,S0004-06142005000900016-1,PROCEDIMIENTO,ct13,gammagrafía renal,739,756
3,S0004-06142005001000011-1,PROCEDIMIENTO,3e1m39z,diálisis peritoneal,95,114
7,S0004-06142005001000011-1,PROCEDIMIENTO,0270,angioplastia transluminal de la coronaria derecha,424,473


In [43]:
df_codes_dev_ner.shape

(1540, 6)

Now, using the character start-end positions of each sentence from the CodiEsp corpus (see `datasets/CodiEsp-Sentence-Split.ipynb`), we annotate the sentences with CIE-Procedimiento codes. Also, using XLM-R tokenizer, each sentence is converted into a sequence of subwords, which are further converted into vocabulary indices (input IDs) and attention mask arrays (XLM-R input tensors). We also generate a *fragments* dataset indicating the number of produced annotated sentences for each document.

In [44]:
# Sentence-Split information
ss_corpus_path = "../datasets/CodiEsp-SSplit-text/"

### Training corpus

In [45]:
label_list = list(df_codes_train_dev["code"])

In [46]:
len(label_list)

2367

In [47]:
len(set(label_list))

727

In [48]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb_encoder = MultiLabelBinarizer()
mlb_encoder.fit([label_list])

MultiLabelBinarizer()

In [49]:
# Number of distinct codes
num_labels = len(mlb_encoder.classes_)

In [50]:
num_labels

727

Only training texts that are annotated with CIE-Procedimiento codes are considered:

In [51]:
# Some train documents (texts) are not annotated 
len(set(df_text_train["doc_id"]) - set(df_codes_train_ner["doc_id"]))

65

In [52]:
train_doc_list = sorted(set(df_codes_train_ner["doc_id"]))

In [53]:
len(train_doc_list)

435

In [54]:
# Sentence-Split data

In [55]:
%%time
ss_sub_corpus_path = ss_corpus_path + "train/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_train = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 4.21 ms, sys: 11.9 ms, total: 16.1 ms
Wall time: 15.9 ms


In [56]:
%%time
train_ind, train_att, train_y, train_frag, train_start_end_frag = ss_create_frag_input_data_xlmr(df_text=df_text_train, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_train_ner, doc_list=train_doc_list, ss_dict=ss_dict_train,
                                                  tokenizer=tokenizer, sp_pb2=spt, lab_encoder=mlb_encoder, seq_len=SEQ_LEN)

100%|██████████| 435/435 [00:04<00:00, 93.91it/s] 


CPU times: user 4.75 s, sys: 18.9 ms, total: 4.77 s
Wall time: 4.74 s


In [57]:
# Sanity check

In [58]:
train_ind.shape

(7013, 128)

In [59]:
train_att.shape

(7013, 128)

In [60]:
train_y.shape

(7013, 727)

In [61]:
len(train_frag)

435

In [62]:
len(train_start_end_frag)

7013

In [63]:
# Check n_frag distribution across texts
pd.Series(train_frag).describe()

count    435.000000
mean      16.121839
std        7.762687
min        4.000000
25%       10.500000
50%       15.000000
75%       20.000000
max       54.000000
dtype: float64

In [64]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(train_doc_list), size=1)[0]

In [65]:
check_id

104

In [66]:
train_doc_list[check_id]

'S0211-69952012000700031-1'

In [67]:
df_text_train[df_text_train["doc_id"] == train_doc_list[check_id]][text_col].values[0]

'Varón de 53 años, jubilado, que vive en área rural. Fue diagnosticado hace 10 años de enfermedad inflamatoria intestinal, que se controla únicamente con dosis altas de esteroides. Dos años antes había ingresado en Unidad de Vigilancia Intensiva por sepsis de origen intestinal con fracaso multiorgánico; desde entonces está en hemodiálisis periódica al no haber recuperación funcional. La biopsia renal mostraba nefropatía tubulointersticial.\nDesde un mes antes del ingreso presenta lesiones cutáneas, eritematosas, sobre zonas induradas, pruriginosas y dolorosas, en dorso del pie derecho y cara interna de los muslos, con mala respuesta al tratamiento antibiótico (iniciado de forma empírica por sospecha de celulitis), que fueron evolucionando hacia la descamación y posterior ulceración. Se realizó biopsia cutánea que es informada como criptococosis. Al examen directo con tinta china y tinción de Gram se observan abundantes levaduras esféricas, de gran tamaño, con gemación marcada y encapsu

In [68]:
check_id_frag = sum(train_frag[:check_id])
for i in range(check_id_frag, check_id_frag + train_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([train_y[i]])), "\n")

[()] 

[()] 

[('5a1d',)] 

[()] 

[('0hbmxzx',)] 

[('0hbmxzx',)] 

[()] 

[()] 

[()] 

[('009u3zx',)] 

[()] 

[()] 

[()] 



In [69]:
for i in range(check_id_frag, check_id_frag + train_frag[check_id]):
    print(list(zip([tokenizer._convert_id_to_token(int(ind)) for ind in train_ind[i]][1:len(train_start_end_frag[i])+1], 
               train_start_end_frag[i])))
    print("\n")

[('▁Var', (0, 3)), ('ón', (3, 6)), ('▁de', (6, 9)), ('▁53', (9, 12)), ('▁años', (12, 18)), (',', (18, 19)), ('▁jubila', (19, 26)), ('do', (26, 28)), (',', (28, 29)), ('▁que', (29, 33)), ('▁vive', (33, 38)), ('▁en', (38, 41)), ('▁área', (41, 47)), ('▁rural', (47, 53)), ('.', (53, 54))]


[('▁Fue', (55, 58)), ('▁diagnostic', (58, 69)), ('ado', (69, 72)), ('▁hace', (72, 77)), ('▁10', (77, 80)), ('▁años', (80, 86)), ('▁de', (86, 89)), ('▁enfermedad', (89, 100)), ('▁inflama', (100, 108)), ('toria', (108, 113)), ('▁intestinal', (113, 124)), (',', (124, 125)), ('▁que', (125, 129)), ('▁se', (129, 132)), ('▁control', (132, 140)), ('a', (140, 141)), ('▁únicamente', (141, 153)), ('▁con', (153, 157)), ('▁dosis', (157, 163)), ('▁alta', (163, 168)), ('s', (168, 169)), ('▁de', (169, 172)), ('▁e', (172, 174)), ('steroid', (174, 181)), ('es', (181, 183)), ('.', (183, 184))]


[('▁Dos', (185, 188)), ('▁años', (188, 194)), ('▁antes', (194, 200)), ('▁había', (200, 207)), ('▁in', (207, 210)), ('gres', (210

In [70]:
check_id_frag = sum(train_frag[:check_id])
for frag in train_ind[check_id_frag:check_id_frag + train_frag[check_id]]:
    print(' '.join([tokenizer._convert_id_to_token(int(ind)) for ind in frag]), "\n")

<s> ▁Var ón ▁de ▁53 ▁años , ▁jubila do , ▁que ▁vive ▁en ▁área ▁rural . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

<s> ▁Fue ▁diagnostic ado ▁hace ▁10 ▁años ▁de ▁enfermedad ▁inflama toria ▁intestinal , ▁que ▁se ▁control a ▁únicamente ▁con ▁dosis ▁alta s ▁de ▁e steroid es . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [71]:
# Fragment labels distribution
pd.Series(np.sum(train_y, axis=1)).describe()

count    7013.000000
mean        0.305290
std         0.652715
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000
dtype: float64

### Development corpus

Only development texts that are annotated with CIE-Procedimiento codes are considered:

In [72]:
# Some dev documents (texts) are not annotated 
len(set(df_text_dev["doc_id"]) - set(df_codes_dev_ner["doc_id"]))

28

In [73]:
dev_doc_list = sorted(set(df_codes_dev_ner["doc_id"]))

In [74]:
len(dev_doc_list)

222

In [75]:
%%time
ss_sub_corpus_path = ss_corpus_path + "dev/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_dev = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 8.98 ms, sys: 0 ns, total: 8.98 ms
Wall time: 8.67 ms


In [76]:
%%time
dev_ind, dev_att, dev_y, dev_frag, dev_start_end_frag = ss_create_frag_input_data_xlmr(df_text=df_text_dev, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_dev_ner, doc_list=dev_doc_list, ss_dict=ss_dict_dev,
                                                  tokenizer=tokenizer, sp_pb2=spt, lab_encoder=mlb_encoder, seq_len=SEQ_LEN)

100%|██████████| 222/222 [00:02<00:00, 85.15it/s]


CPU times: user 2.67 s, sys: 11.7 ms, total: 2.68 s
Wall time: 2.67 s


In [77]:
# Sanity check

In [78]:
dev_ind.shape

(3799, 128)

In [79]:
dev_att.shape

(3799, 128)

In [80]:
dev_y.shape

(3799, 727)

In [81]:
len(dev_frag)

222

In [82]:
len(dev_start_end_frag)

3799

In [83]:
# Check n_frag distribution across texts
pd.Series(dev_frag).describe()

count    222.000000
mean      17.112613
std        8.320553
min        4.000000
25%       11.000000
50%       15.000000
75%       21.000000
max       65.000000
dtype: float64

In [84]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(dev_doc_list), size=1)[0]

In [85]:
check_id

9

In [86]:
dev_doc_list[check_id]

'S0004-06142007000300012-1'

In [87]:
df_text_dev[df_text_dev["doc_id"] == dev_doc_list[check_id]][text_col].values[0]

"Paciente de 52 años con antecedentes personales de insuficiencia renal terminal de etiología no filiada que recibió un trasplante heterotópico en FID procedente de donante cadáver en Enero de 1999. Previamente había precisado hemodiálisis desde 1980 a 1983, y diálisis peritoneal desde entonces hasta el momento del trasplante renal.\nNo existieron complicaciones postrasplante. Al paciente se le administró triple terapia inmunosupresora con Ciclosporina, Prednisona y Micofenolato, presentando diuresis inmediata con mejoría progresiva de la función renal, siendo la creatininemia al alta de 1,8 mg/dl.\nDespués de un periodo postrasplante no complicado y asintomático de 51 meses en el que mantuvo una función renal estable (creatininemia: 1,2-1,4 mg/dl) en una ecografía de control se objetivó un nódulo sólido de 1,5 cm vascularizado en el polo inferior del injerto renal. La posterior PAAF ecodirigida reveló que se trataba de un CCR.\n\nEn Junio de 2003 mediante reapertura de la incisión de 

In [88]:
check_id_frag = sum(dev_frag[:check_id])
for i in range(check_id_frag, check_id_frag + dev_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([dev_y[i]])), "\n")

[('0ty0',)] 

[('0ty0', '3e1m39z', '5a1d00z')] 

[()] 

[('0ty0',)] 

[('bt49zzz',)] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[('bt33zzz', 'bt43zzz', 'bt49zzz')] 



In [89]:
for i in range(check_id_frag, check_id_frag + dev_frag[check_id]):
    print(list(zip([tokenizer._convert_id_to_token(int(ind)) for ind in dev_ind[i]][1:len(dev_start_end_frag[i])+1], 
               dev_start_end_frag[i])))
    print("\n")

[('▁Pacient', (0, 7)), ('e', (7, 8)), ('▁de', (8, 11)), ('▁52', (11, 14)), ('▁años', (14, 20)), ('▁con', (20, 24)), ('▁antecede', (24, 33)), ('ntes', (33, 37)), ('▁personales', (37, 48)), ('▁de', (48, 51)), ('▁insu', (51, 56)), ('fici', (56, 60)), ('encia', (60, 65)), ('▁renal', (65, 71)), ('▁terminal', (71, 80)), ('▁de', (80, 83)), ('▁et', (83, 86)), ('i', (86, 87)), ('ología', (87, 94)), ('▁no', (94, 97)), ('▁filia', (97, 103)), ('da', (103, 105)), ('▁que', (105, 109)), ('▁recibió', (109, 118)), ('▁un', (118, 121)), ('▁tra', (121, 125)), ('splan', (125, 130)), ('te', (130, 132)), ('▁hetero', (132, 139)), ('tó', (139, 142)), ('pico', (142, 146)), ('▁en', (146, 149)), ('▁F', (149, 151)), ('ID', (151, 153)), ('▁procede', (153, 161)), ('nte', (161, 164)), ('▁de', (164, 167)), ('▁donant', (167, 174)), ('e', (174, 175)), ('▁cadáver', (175, 184)), ('▁en', (184, 187)), ('▁En', (187, 190)), ('ero', (190, 193)), ('▁de', (193, 196)), ('▁1999.', (196, 202))]


[('▁Prev', (203, 207)), ('i', (207,

In [90]:
check_id_frag = sum(dev_frag[:check_id])
for frag in dev_ind[check_id_frag:check_id_frag + dev_frag[check_id]]:
    print(' '.join([tokenizer._convert_id_to_token(int(ind)) for ind in frag]), "\n")

<s> ▁Pacient e ▁de ▁52 ▁años ▁con ▁antecede ntes ▁personales ▁de ▁insu fici encia ▁renal ▁terminal ▁de ▁et i ología ▁no ▁filia da ▁que ▁recibió ▁un ▁tra splan te ▁hetero tó pico ▁en ▁F ID ▁procede nte ▁de ▁donant e ▁cadáver ▁en ▁En ero ▁de ▁1999. </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

<s> ▁Prev i amente ▁había ▁precisa do ▁hem odi ális is ▁desde ▁1980 ▁a ▁1983 , ▁y ▁di ális is ▁per ito ne al ▁desde ▁entonces ▁hasta ▁el ▁momento ▁del ▁tra splan te ▁renal . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [91]:
# Fragment labels distribution
pd.Series(np.sum(dev_y, axis=1)).describe()

count    3799.000000
mean        0.309818
std         0.638081
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         5.000000
dtype: float64

### Training & Development corpus

We merge the previously generated datasets:

In [92]:
# Indices
train_dev_ind = np.concatenate((train_ind, dev_ind))

In [93]:
train_dev_ind.shape

(10812, 128)

In [94]:
# Attention masks
train_dev_att = np.concatenate((train_att, dev_att))

In [95]:
train_dev_att.shape

(10812, 128)

In [96]:
# y
train_dev_y = np.concatenate((train_y, dev_y))

In [97]:
train_dev_y.shape

(10812, 727)

## Fine-tuning

Using the corpus of labeled sentences, we fine-tune the model on a multi-label sentence classification task.

In [98]:
from transformers import TFXLMRobertaForSequenceClassification

model = TFXLMRobertaForSequenceClassification.from_pretrained(model_name, from_pt=True)

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [99]:
model.summary()

Model: "tfxlm_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  277453056 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  592130    
Total params: 278,045,186
Trainable params: 278,045,186
Non-trainable params: 0
_________________________________________________________________


In [100]:
model.layers

In [101]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.initializers import GlorotUniform

input_ids = Input(shape=(SEQ_LEN,), name='input_ids', dtype='int64')
attention_mask = Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int64')
inputs = [input_ids, attention_mask]

cls_token = model.layers[0](input_ids=inputs[0], attention_mask=inputs[1])[0][:, 0, :] # take <s> token output representation (equiv. to [CLS]) 
out_logits = Dense(units=num_labels, kernel_initializer=GlorotUniform(seed=random_seed))(cls_token) # Multi-label classification
out_act = Activation('sigmoid')(out_logits)

model = Model(inputs=[input_ids, attention_mask], outputs=out_act)

In [102]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
roberta (TFRobertaMainLayer)    TFBaseModelOutputWit 277453056   input_ids[0][0]                  
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 768)]        0           roberta[0][0]                    
______________________________________________________________________________________________

In [103]:
model.input

[<tf.Tensor 'input_ids:0' shape=(None, 128) dtype=int64>,
 <tf.Tensor 'attention_mask:0' shape=(None, 128) dtype=int64>]

In [104]:
model.output

<tf.Tensor 'activation_4/Identity:0' shape=(None, 727) dtype=float32>

In [ ]:
%%time
from tensorflow.keras import optimizers, losses
import tensorflow_addons as tfa

optimizer = tfa.optimizers.RectifiedAdam(learning_rate=LR)
loss = losses.BinaryCrossentropy(from_logits=False)
model.compile(optimizer=optimizer, loss=loss)

history = model.fit(x={'input_ids': train_dev_ind, 'attention_mask': train_dev_att}, y=train_dev_y,
          batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True)

Epoch 1/41
676/676 [==============================] - 172s 255ms/step - loss: 0.2379
Epoch 2/41
676/676 [==============================] - 174s 257ms/step - loss: 0.0208
Epoch 3/41
676/676 [==============================] - 174s 257ms/step - loss: 0.0075
Epoch 4/41
676/676 [==============================] - 174s 257ms/step - loss: 0.0048
Epoch 5/41
676/676 [==============================] - 174s 257ms/step - loss: 0.0040
Epoch 6/41
676/676 [==============================] - 174s 257ms/step - loss: 0.0036
Epoch 7/41
676/676 [==============================] - 174s 258ms/step - loss: 0.0034
Epoch 8/41
676/676 [==============================] - 174s 258ms/step - loss: 0.0032
Epoch 9/41
676/676 [==============================] - 174s 258ms/step - loss: 0.0031
Epoch 10/41
676/676 [==============================] - 174s 258ms/step - loss: 0.0029
Epoch 11/41
676/676 [==============================] - 174s 258ms/step - loss: 0.0027
Epoch 12/41
676/676 [==============================] - 174s 258

## Test set predictions

Finally, the predictions made by the model on the test set are saved. For this purpose, firstly, each sentence from the test corpus must be converted into a sequence of subwords (input IDs and attention mask arrays). Then, the predictions made by the model at the sentence-level are saved, to be further evaluated at document-level (see `results/CodiEsp-P/Evaluation.ipynb`).

In [98]:
%%time
test_path = corpus_path + "test/text_files/"
test_files = [f for f in os.listdir(test_path) if os.path.isfile(test_path + f)]
test_data = load_text_files(test_files, test_path)
df_text_test = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in test_files], 'raw_text': test_data})

CPU times: user 6.74 ms, sys: 0 ns, total: 6.74 ms
Wall time: 6.07 ms


In [99]:
df_text_test.shape

(250, 2)

In [100]:
df_text_test.head()

,doc_id,raw_text
0,S0365-66912007000900014-1,Paciente varón de 34 años de edad diagnosticad...
1,S0211-69952014000200012-1,"Un varón de 48 años, de raza caucásica, con IR..."
2,S1139-76322017000200009-1,Presentamos el caso clínico de un niño de cinc...
3,S0210-48062010000100019-1,"Paciente varón de 53 años, diagnosticado de es..."
4,S1130-14732005000500006-1,Se trata de un varón de 20 años diagnosticado ...


In [101]:
df_text_test.raw_text[0]

'Paciente varón de 34 años de edad diagnosticado de varicela tres semanas antes ya resuelta sin complicaciones. Acude a urgencias por presentar disminución de agudeza visual en su ojo izquierdo.\nEn la exploración oftalmológica presenta una agudeza visual corregida de 1 en el ojo derecho (OD) y de 0,6 en el ojo izquierdo (OI). El estudio con lámpara de hendidura demuestra en el OI un tyndall celular de 4+, precipitados queráticos inferiores (3+) y sin presentar la cornea tinción con fluoresceína, siendo normal el OD. La presión intraocular fue de 16mmHg en ambos ojos.\nEn la exploración fundoscópica inicial del OI se aprecia leve vitritis (1+) sin focos de retinitis.\nSe instaura tratamiento tópico con corticoides y midriáticos. A los 2 días se observa leve disminución del tyndall celular (3+) en cámara anterior pero en fondo de ojo aparece un foco periférico de retinitis necrotizante en el área temporal asociado a vasculitis retiniana.\nSe ingresa al paciente y se instaura tratamiento

In [102]:
test_doc_list = sorted(set(df_text_test["doc_id"]))

In [103]:
len(test_doc_list)

250

In [111]:
%%time
ss_sub_corpus_path = ss_corpus_path + "test/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_test = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 34.7 ms, sys: 0 ns, total: 34.7 ms
Wall time: 34.1 ms


In [112]:
%%time
test_ind, test_att, _, test_frag, _ = ss_create_frag_input_data_xlmr(df_text=df_text_test, 
                                                  text_col=text_col,
                                                  # Since labels are ignored, we pass df_codes_train_ner as df_ann
                                                  df_ann=df_codes_train_ner, doc_list=test_doc_list, ss_dict=ss_dict_test,
                                                  tokenizer=tokenizer, sp_pb2=spt, lab_encoder=mlb_encoder, seq_len=SEQ_LEN)

100%|██████████| 250/250 [00:00<00:00, 279.35it/s]


CPU times: user 956 ms, sys: 12.4 ms, total: 968 ms
Wall time: 951 ms


In [113]:
%%time
test_preds = model.predict({'input_ids': test_ind, 'attention_mask': test_att})

CPU times: user 14.9 s, sys: 1.68 s, total: 16.6 s
Wall time: 16.9 s


In [114]:
test_preds.shape

(3950, 727)

In [104]:
results_dir_path = "../results/CodiEsp-P/"

In [178]:
%%time
np.save(file=results_dir_path + "predictions/xlm_r_seed_" + str(random_seed) + "_test_preds.npy", arr=test_preds)

CPU times: user 2.21 ms, sys: 4.65 ms, total: 6.87 ms
Wall time: 6.02 ms


In [111]:
# To be further used when evaluating model performance at document level
np.save(file=results_dir_path + "xlm_r_test_frags.npy", arr=test_frag)
np.save(file=results_dir_path + "classes.npy", arr=mlb_encoder.classes_)
np.save(file=results_dir_path + "test_docs.npy", arr=test_doc_list)